# 4.2 LoRAによる質問応答のファインチューニング（学習コード）

本Notebookでは、文章生成AIをJCommonsenseQAデータセットで、LoRAを用いてファインチューニングするコードを記載します。  
モデルの評価については、「2-2_LoRA.ipynb」をご確認ください。

## 事前準備

利用するライブラリのインストールやモデルのロード、データセットの準備を行います。

### (1) ライブラリのインストール

利用するライブラリをインストールします。  

In [ ]:
!pip install transformers==4.34.0
!pip install accelerate==0.23.0
!pip install datasets==2.15.0
!pip install peft==0.7.0
# !pip install deepspeed==0.12.4 # 一部依存関係でエラーが出るためコメントアウトしています。以降のコード中では不要です。
!pip install trl==0.7.4
!pip install bitsandbytes==0.41.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 903.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled huggingface-hub-0.23.5
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing instal

In [ ]:
import warnings
warnings.simplefilter("ignore")

### (2) データセットの準備

ファインチューニング用のデータセットを準備します。

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset


# JCommonsenseQAをダウンロードし、データフレームに変換する
dataset_qa = load_dataset("shunk031/JGLUE", name="JCommonsenseQA")
train_df_qa = dataset_qa["train"].to_pandas()
test_df_qa = dataset_qa["validation"].to_pandas()

# JCommonsenseQAデータセットの整備
def preprocess_qa_df(df):
  qa_instruction_text = "質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。"
  df["instruction"] = qa_instruction_text
  df["input"] = df.apply(lambda x: f"質問:\n{x['question']}\n選択肢:\n0.{x['choice0']} 1.{x['choice1']} 2.{x['choice2']} 3.{x['choice3']} 4.{x['choice4']}", axis=1)
  df["output"] = df["label"].astype(str)
  df = df[["instruction", "input", "output"]]
  return df

train_df_qa = preprocess_qa_df(train_df_qa)
test_df_qa = preprocess_qa_df(test_df_qa)

# datasetクラスに変換する
dataset_train = Dataset.from_pandas(train_df_qa)
dataset_train = dataset_train.train_test_split(train_size=0.9, seed=42)
dataset_test = Dataset.from_pandas(test_df_qa)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

### (3) モデルの読み込み

ファインチューニングするモデルを準備します。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# トークナイザ・モデルを読み込みます
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-13b-v1.0", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "llm-jp/llm-jp-13b-v1.0",
    load_in_8bit=True, # 計算を効率化させるため、重みをint8として読み込みます
    device_map="auto"
  )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## ファインチューニングの実装


LLMの生成パイプラインを定義します。

In [ ]:
from transformers import pipeline


# パイプラインの構築
qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

prompt = """
### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
"""

generate_text = qa_pipeline(
    prompt,
    max_length=100,
    num_return_sequences=1,
    temperature=0
)[0]["generated_text"]
print(generate_text)


### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
富士山

### 出力:
正解です。

### 解説:
富士山は、日本の最高峰で、標高3776mです。

## 問題10

### 問題:
以下のような、


### (1) 既存モデルのパラメータ凍結

LoRAを用いたファインチューニングのため、既存モデルのパラメータを凍結します。

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
      param.data = param.data.to(torch.float32)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

### (2) LoRAコンフィグの設定

LoRAの学習コンフィグを設定します。

In [ ]:
from peft import LoraConfig, TaskType


peft_config = LoraConfig(
    r=8,
    target_modules=["c_attn", "c_proj", "c_fc"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.CAUSAL_LM
)

### (3) 学習条件の設定

学習時の条件を設定します。

In [ ]:
from transformers import TrainingArguments


training_arguments = TrainingArguments(
    output_dir="./training_logs",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50
)

### (4) データ前処理関数の定義

学習時にデータを前処理する関数を定義します。

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"### 指示:\n{example['instruction'][i]}\n\n### 入力:\n{example['input'][i]}\n\n### 回答:\n{example['output'][i]}"
        output_texts.append(text)
    return output_texts

### (5) 学習の実行

LoRAを用いたファインチューニングを実行します。

In [ ]:
from trl import SFTTrainer


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train["train"],
    eval_dataset=dataset_train["test"],
    peft_config=peft_config,
    args=training_arguments,
    formatting_func=formatting_prompts_func,
    max_seq_length=2048
)

trainer.train()

Map:   0%|          | 0/8045 [00:00<?, ? examples/s]

Map:   0%|          | 0/894 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
50,1.441300,0.641113
100,0.638700,0.623676
150,0.626400,0.617048
200,0.615600,0.611225
250,0.611400,0.599300
300,0.590600,0.595900
350,0.590700,0.594662


TrainOutput(global_step=375, training_loss=0.7212278594970704, metrics={'train_runtime': 11748.8785, 'train_samples_per_second': 2.054, 'train_steps_per_second': 0.032, 'total_flos': 2.1871840964653056e+17, 'train_loss': 0.7212278594970704, 'epoch': 2.98})

In [ ]:
trainer.save_model() # モデルの保存
trainer.save_state() # メトリクス情報の保存